### chatty 프로그램 로그데이터 처리용 코드입니다.

In [ ]:
import pandas as pd

# 열 정의
columns = ['timeline',
           'id',
           'streamer',
           'moderator',
           'subscriber',
           'twitch_prime',
           'cheer',
           'chat']

# 전처리 함수
def preprocessor(line):
    if line[0] != '[':  # '['로 시작되지 않으면 건너뜀
        return(0)
    if line.find('<') == -1:  # id를 감싸고 있는 '<'가 없으면 건너뜀
        return(0)
    
    timeline_start = 1  # timeline이 시작되는 지점의 index
    timeline_end = line.find(']')  # timeline이 끝나는 지점의 index
    id_start = line.find('<') + 1  # id 부분이 시작되는 지점의 index
    id_end = line.find('>')  # id 부분이 끝나는 지점의 index
    
    # 타임라인 처리
    timeline = pd.to_datetime(line[timeline_start:timeline_end])

    # id 처리
    id_container = line[id_start:id_end]
    if id_container.find('(') > -1:
        id_ = id_container[id_container.find('(')+1 : -1]
    else:
        id_ = id_container\
        .replace('~', '')\
        .replace('@', '')\
        .replace('%', '')\
        .replace('+', '')\
        .replace('$', '')

    # badge 마크 처리
    if '~' in id_container:
        streamer = True
    else:
        streamer = False
    if '@' in id_container:
        moderator = True
    else:
        moderator = False
    if '%' in id_container:
        subscriber = True
    else:
        subscriber = False
    if '+' in id_container:
        twitch_prime = True
    else:
        twitch_prime = False
    if '$' in id_container:
        cheer = True
    else:
        cheer = False

    # chat 처리
    chat = line[line.find('>')+2 : -1]
    
    return([timeline, id_, streamer, moderator, subscriber, twitch_prime, cheer, chat])

In [ ]:
# 데이터 읽기
import os

path_dir = './data/video_chat/'
file_list = os.listdir(path_dir)

lines = []
for fl in file_list:
    if fl.find('.log') is not -1:
        with open(path_dir + fl, 'r', encoding='utf-8') as f:
            lines = lines + f.readlines()
        print('{}: {} lines'.format(fl, len(lines)))

In [ ]:
# csv 파일 생성
with open('./data/video_chat/chat_log.csv', 'w', encoding='utf-8') as f:
    for column in columns:
        f.write(column)
        if column != columns[-1]:
            f.write(',')
        else:
            f.write('\n')

data_size = 0
batch_size = 500
epoch_size = int(len(lines) / batch_size)

print('batch_size: {}, epoch_size: {}'.format(batch_size, epoch_size + 1))
for epoch in range(epoch_size + 1):
    if epoch < epoch_size + 1:
        batch = lines[epoch*batch_size : (epoch+1)*batch_size]
    else:
        if len(lines) % batch_size == 0:
            break
        else:
            batch = lines[epoch*batch_size : (epoch)*batch_size + len(lines)%batch_size]

    # 데이터 프레임 생성
    df = pd.DataFrame(columns=columns)

    # 배치 크기만큼 전처리 후 데이터 프레임에 저장
    for idx, line in enumerate(batch):
        labels = preprocessor(line)
        if labels == 0:
            continue
        else:
            for i, item in enumerate(labels):
                df.loc[idx, columns[i]] = item
    
    # 배치 데이터 csv 파일에 추가
    df.to_csv("./data/video_chat/chat_log.csv", mode='a', index=False, header=False)
    data_size = data_size + df.shape[0]
    print('epoch: {}/{}, data_size: {}'.format(epoch + 1, epoch_size + 1, data_size))
    
print('done')